In [1]:
!pip install networkx plotly



In [2]:
import pandas as pd
import re
import networkx as nx
import plotly.graph_objects as go

In [3]:
import pandas as pd
import re
import networkx as nx
import plotly.graph_objects as go

# Define the file path for your data
file_path = '/content/Task10_Gene_to_Chemical.csv'

# Read the CSV file into a dataframe
df = pd.read_csv(file_path)

def preprocess_sentence(sentence):
    # Remove punctuation and convert to lowercase
    sentence = re.sub(r'[^\w\s]', '', sentence).lower()
    return sentence

# Convert 'Sentence' column to string
df['Sentence'] = df['Sentence'].astype(str)

# Preprocess sentences in the dataframe
df['Sentence'] = df['Sentence'].apply(preprocess_sentence)

# Select only 500 rows to show the result quickly
df_top_500 = df.head(500)

# Create a directed graph
G = nx.DiGraph()

# Add edges (connections) between chemicals and genes with interaction type as edge attribute
for _, row in df_top_500.iterrows():
    G.add_edge(row['ChemicalName'], row['Genes'], interaction_type=row.get('Interaction type', 'Unknown'))

# Create a 3D spring layout
pos = nx.spring_layout(G, dim=3)

# Extract node positions for 3D plotting
node_positions = {node: pos[node] for node in G.nodes}

# Create edge trace for plotting
edge_x = []
edge_y = []
edge_z = []
edge_text = []
for edge in G.edges(data=True):
    x0, y0, z0 = node_positions[edge[0]]
    x1, y1, z1 = node_positions[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])
    edge_z.extend([z0, z1, None])
    edge_text.append(edge[2].get('interaction_type', 'Unknown'))

# Create node trace for plotting
node_x = []
node_y = []
node_z = []
node_text = []
for node in G.nodes():
    x, y, z = node_positions[node]
    node_x.append(x)
    node_y.append(y)
    node_z.append(z)
    node_text.append(node)

# Create 3D plot
fig = go.Figure()

# Add edges
fig.add_trace(go.Scatter3d(
    x=edge_x, y=edge_y, z=edge_z,
    mode='lines',
    line=dict(color='rgb(125,125,125)', width=1),
    hoverinfo='text',
    text=edge_text,
    name='Interactions'
))

# Add nodes
fig.add_trace(go.Scatter3d(
    x=node_x, y=node_y, z=node_z,
    mode='markers+text',
    marker=dict(symbol='circle', size=5, color='rgb(0,0,255)'),
    text=node_text,
    textposition="top center",
    hoverinfo='text',
    hovertext=node_text,
    name='Nodes'
))

# Set layout
fig.update_layout(
    title='Gene-Chemical Interaction Knowledge Graph (Top 500)',
    showlegend=True,
    scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False),
    )
)

# Show the plot
fig.show()

# Save the plot as an HTML file
fig.write_html('/content/gene-Chemical_interaction_knowledge_graph.html')


### **To show the nodes functioaility**

In [4]:
import pandas as pd
import re
import networkx as nx
import plotly.graph_objects as go

# Define the file paths for your data
chemicals_file_path = '/content/Task3_Secondary_Metabolites_Bacteria._Chemicals_pubmed.csv'
genes_file_path = '/content/Task3_Secondary_Metabolites_Bacteria._Genes_pubmed.csv'
interactions_file_path = '/content/Task10_Gene_to_Chemical.csv'

# Read the chemical, gene, and interactions CSV files into dataframes
chemicals_df = pd.read_csv(chemicals_file_path)
genes_df = pd.read_csv(genes_file_path)
interactions_df = pd.read_csv(interactions_file_path)

def preprocess_sentence(sentence):
    # Remove punctuation and convert to lowercase
    sentence = re.sub(r'[^\w\s]', '', sentence).lower()
    return sentence

# Convert 'Name' column in chemicals_df and 'Gene_Name' column in genes_df to string and preprocess
chemicals_df['Chemical'] = chemicals_df['Chemical'].astype(str).apply(preprocess_sentence)
genes_df['Gene'] = genes_df['Gene'].astype(str).apply(preprocess_sentence)

# Preprocess sentences in the interactions dataframe
interactions_df['Sentence'] = interactions_df['Sentence'].astype(str).apply(preprocess_sentence)

# i am selecting only 500 to show the result in a quick manner
interactions_df_top_100 = interactions_df.head(500)

# Create a directed graph
G = nx.DiGraph()

# Add edges (connections) between chemicals and genes with interaction type as edge attribute
for _, row in interactions_df_top_100.iterrows():
    G.add_edge(row['ChemicalName'], row['Genes'], interaction_type=row['Interaction type'])

# Create dictionaries to store information about chemicals and genes
chemical_info = {name: (chem_id, length, pmid) for chem_id, name, length, pmid in zip(
    chemicals_df['ChemicalID'], chemicals_df['Chemical'], chemicals_df['length'], chemicals_df['PMID_List']
)}
gene_info = {name: (gene_id, length, pmid) for gene_id, name, length, pmid in zip(
    genes_df['GeneID'], genes_df['Gene'], genes_df['length'], genes_df['PMID_List']
)}

# Create 3D spring layout
pos = nx.spring_layout(G, dim=3)

# Extract node positions for 3D plotting
node_positions = {node: pos[node] for node in G.nodes}

# Create edge trace for plotting
edge_x = []
edge_y = []
edge_z = []
edge_text = []
for edge in G.edges(data=True):
    x0, y0, z0 = node_positions[edge[0]]
    x1, y1, z1 = node_positions[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])
    edge_z.extend([z0, z1, None])
    edge_text.append(edge[2]['interaction_type'])

# Create node trace for plotting
node_x = []
node_y = []
node_z = []
node_text = []
node_hover_text = []
for node in G.nodes():
    x, y, z = node_positions[node]
    node_x.append(x)
    node_y.append(y)
    node_z.append(z)
    node_text.append(node)
    if node in chemical_info:
        info_str = f"Chemical ID: {chemical_info[node][0]}<br>" \
                   f"Length: {chemical_info[node][1]}<br>" \
                   f"PMID: {chemical_info[node][2]}"
    elif node in gene_info:
        info_str = f"Gene ID: {gene_info[node][0]}<br>" \
                   f"Length: {gene_info[node][1]}<br>" \
                   f"PMID: {gene_info[node][2]}"
    else:
        info_str = "No information available"
    node_hover_text.append(info_str)

# Create 3D plot
fig = go.Figure()

# Add edges
fig.add_trace(go.Scatter3d(
    x=edge_x, y=edge_y, z=edge_z,
    mode='lines',
    line=dict(color='rgb(125,125,125)', width=1),
    hoverinfo='text',
    text=edge_text,
    name='Interactions'
))

# Add nodes
fig.add_trace(go.Scatter3d(
    x=node_x, y=node_y, z=node_z,
    mode='markers+text',
    marker=dict(symbol='circle', size=5, color='rgb(0,0,255)'),
    text=node_text,
    textposition="top center",
    hoverinfo='text',
    hovertext=node_hover_text,
    name='Nodes'
))

# Set layout
fig.update_layout(
    title='Gene-Chemical Interaction Knowledge Graph_with_information (Top 500)',
    showlegend=True,
    scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False),
    ),
    hovermode='closest',  # Show hover info closest to the mouse
)

# Add click event handling for nodes
fig.update_traces(marker=dict(size=10),
                  selector=dict(mode='markers+text'),
                  textposition='top center')

fig.show()
# Save the plot as an HTML file
fig.write_html('/content/gene-Chemical_interaction_knowledge_graph(with information).html')


In [5]:
!pip install streamlit


In [6]:
import streamlit as st
import pandas as pd
import re
import networkx as nx
import plotly.graph_objects as go

# Define the file paths for your data
chemicals_file_path = '/content/Task3_Secondary_Metabolites_Bacteria._Chemicals_pubmed.csv'
genes_file_path = '/content/Task3_Secondary_Metabolites_Bacteria._Genes_pubmed.csv'
interactions_file_path = '/content/Task10_Gene_to_Chemical.csv'

# Read the chemical, gene, and interactions CSV files into dataframes
chemicals_df = pd.read_csv(chemicals_file_path)
genes_df = pd.read_csv(genes_file_path)
interactions_df = pd.read_csv(interactions_file_path, encoding='utf-8')

def preprocess_sentence(sentence):
    # Remove punctuation and convert to lowercase
    sentence = re.sub(r'[^\w\s]', '', sentence).lower()
    return sentence

# Convert 'Name' column in chemicals_df and 'Gene_Name' column in genes_df to string and preprocess
chemicals_df['Chemical'] = chemicals_df['Chemical'].astype(str).apply(preprocess_sentence)
genes_df['Gene'] = genes_df['Gene'].astype(str).apply(preprocess_sentence)

# Preprocess sentences in the interactions dataframe
interactions_df['Sentence'] = interactions_df['Sentence'].astype(str).apply(preprocess_sentence)

# i am selecting only 500 to show the result in a quick manner
interactions_df_top_100 = interactions_df.head(500)

# Create a directed graph
G = nx.DiGraph()

# Add edges (connections) between chemicals and genes with interaction type as edge attribute
for _, row in interactions_df_top_100.iterrows():
    G.add_edge(row['ChemicalName'], row['Genes'], interaction_type=row['Interaction type'])

# Create dictionaries to store information about chemicals and genes
chemical_info = {name: (chem_id, length, pmid) for chem_id, name, length, pmid in zip(
    chemicals_df['ChemicalID'], chemicals_df['Chemical'], chemicals_df['length'], chemicals_df['PMID_List']
)}
gene_info = {name: (gene_id, length, pmid) for gene_id, name, length, pmid in zip(
    genes_df['GeneID'], genes_df['Gene'], genes_df['length'], genes_df['PMID_List']
)}

# Create 3D spring layout
pos = nx.spring_layout(G, dim=3)

# Extract node positions for 3D plotting
node_positions = {node: pos[node] for node in G.nodes}

# Create a Streamlit app
st.title('Gene-Chemical Interaction Graph')

# Select a node
selected_node = st.selectbox('Select a node:', list(G.nodes()))

# Create edge trace for plotting
edge_x = []
edge_y = []
edge_z = []
edge_text = []
for edge in G.edges(data=True):
    x0, y0, z0 = node_positions[edge[0]]
    x1, y1, z1 = node_positions[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])
    edge_z.extend([z0, z1, None])
    edge_text.append(edge[2]['interaction_type'])

# Create node trace for plotting
node_x = []
node_y = []
node_z = []
node_text = []
node_hover_text = []
for node in G.nodes():
    x, y, z = node_positions[node]
    node_x.append(x)
    node_y.append(y)
    node_z.append(z)
    node_text.append(node)
    if node in chemical_info:
        info_str = f"Chemical ID: {chemical_info[node][0]}"
        info_str += f"<br>Length: {chemical_info[node][1]}"
        info_str += f"<br>PMID: {chemical_info[node][2]}"
    elif node in gene_info:
        info_str += f"Gene ID: {gene_info[node][0]}"
        info_str += f"<br>Length: {gene_info[node][1]}"
        info_str += f"<br>PMID: {gene_info[node][2]}"
    else:
        info_str += "No information available"
    node_hover_text.append(info_str)

# Filter the graph based on the selected node
filtered_nodes = [selected_node] + list(G.neighbors(selected_node))
filtered_edges = [(source, target) for source, target in G.edges() if source in filtered_nodes and target in filtered_nodes]

# Update node and edge traces for plotting
filtered_node_x = [node_x[node_text.index(node)] for node in filtered_nodes]
filtered_node_y = [node_y[node_text.index(node)] for node in filtered_nodes]
filtered_node_z = [node_z[node_text.index(node)] for node in filtered_nodes]
filtered_node_text = [node_text[node_text.index(node)] for node in filtered_nodes]
filtered_node_hover_text = [node_hover_text[node_text.index(node)] for node in filtered_nodes]

filtered_edge_x = []
filtered_edge_y = []
filtered_edge_z = []
filtered_edge_text = []
for edge in filtered_edges:
    x0, y0, z0 = node_positions[edge[0]]
    x1, y1, z1 = node_positions[edge[1]]
    filtered_edge_x.extend([x0, x1, None])
    filtered_edge_y.extend([y0, y1, None])
    filtered_edge_z.extend([z0, z1, None])
    filtered_edge_text.append(G[edge[0]][edge[1]]['interaction_type'])

# Create 3D plot
fig = go.Figure()

# Add edges
fig.add_trace(go.Scatter3d(
    x=filtered_edge_x, y=filtered_edge_y, z=filtered_edge_z,
    mode='lines',
    line=dict(color='rgb(125,125,125)', width=1),
    hoverinfo='text',
    text=filtered_edge_text,
    name='Interactions'
))

# Add nodes
fig.add_trace(go.Scatter3d(
    x=filtered_node_x, y=filtered_node_y, z=filtered_node_z,
    mode='markers+text',
    marker=dict(symbol='circle', size=5, color='rgb(0,0,255)'),
    text=filtered_node_text,
    textposition="top center",
    hoverinfo='text',
    hovertext=filtered_node_hover_text,
    name='Nodes'
))

# Set layout
fig.update_layout(
    title=f'Gene-Chemical Interaction Graph (Node: {selected_node})',
    showlegend=True,
    scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False),
    ),
    hovermode='closest',  # Show hover info closest to the mouse
)

# Add click event handling for nodes
fig.update_traces(marker=dict(size=10),
                  selector=dict(mode='markers+text'),
                  textposition='top center')

# Display the plot
st.plotly_chart(fig)
fig.show()





2024-08-02 04:20:05.346 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-08-02 04:20:05.354 Session state does not function when running a script without `streamlit run`


In [7]:
!pip install streamlit


In [8]:
!streamlit run --server.enableCORS false --server.port PORT_NUMBER app.py &>/dev/null&


In [9]:
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip


In [10]:
get_ipython().system_raw('./ngrok http PORT_NUMBER &')


In [11]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.10/json/__init__.py", line 293, in load
    return loads(fp.read(),
  File "/usr/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/lib/python3.10/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


In [12]:
import requests
from bs4 import BeautifulSoup

ngrok_tunnel_url = None

# Fetch ngrok tunnel URL
try:
    response = requests.get('http://localhost:4040/api/tunnels')
    data = response.json()
    ngrok_tunnel_url = data['tunnels'][0]['public_url']
except Exception as e:
    print(f"Error fetching ngrok URL: {e}")

ngrok_tunnel_url


Error fetching ngrok URL: HTTPConnectionPool(host='localhost', port=4040): Max retries exceeded with url: /api/tunnels (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c91bc0ef070>: Failed to establish a new connection: [Errno 111] Connection refused'))


In [13]:
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip
# get_ipython().system_raw('./ngrok http PORT_NUMBER &')


--2024-08-02 04:20:11--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 54.161.241.46, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.7’

ngrok-stable-linux- 100%[===================>]  13.28M  19.2MB/s    in 0.7s    

2024-08-02 04:20:12 (19.2 MB/s) - ‘ngrok-stable-linux-amd64.zip.7’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [24]:
import pandas as pd
import re
import networkx as nx
import plotly.graph_objects as go

# Define the file path for your data
file_path = '/content/Task10_Gene_to_Chemical.csv'

# Read the CSV file into a dataframe
df = pd.read_csv(file_path)

def preprocess_sentence(sentence):
    # Remove punctuation and convert to lowercase
    sentence = re.sub(r'[^\w\s]', '', sentence).lower()
    return sentence

# Convert 'Sentence' column to string
df['Sentence'] = df['Sentence'].astype(str)

# Preprocess sentences in the dataframe
df['Sentence'] = df['Sentence'].apply(preprocess_sentence)

# I am selecting only 1160 to show the result quickly
df_top_100 = df.head(1160)

# Create a directed graph
G = nx.DiGraph()

# Add edges (connections) between chemicals and genes with interaction type as edge attribute
for _, row in df_top_100.iterrows():
    G.add_edge(row['ChemicalName'], row['Genes'], interaction_type=row['Interaction type'])

# Create a 3D spring layout
pos = nx.spring_layout(G, dim=3)

# Extract node positions for 3D plotting
node_positions = {node: pos[node] for node in G.nodes}

# Create edge trace for plotting
edge_x = []
edge_y = []
edge_z = []
edge_text = []
edge_color = []  # New list for edge colors based on interaction type

for edge in G.edges(data=True):
    x0, y0, z0 = node_positions[edge[0]]
    x1, y1, z1 = node_positions[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])
    edge_z.extend([z0, z1, None])
    edge_text.append(edge[2]['interaction_type'])

    # Assign edge colors based on interaction type
    if edge[2]['interaction_type'] == 'Proliferation Inhibition':
        edge_color.extend(['orange', 'orange', 'orange'])
    elif edge[2]['interaction_type'] == 'Activation':
        edge_color.extend(['pink', 'pink', 'pink'])
    elif edge[2]['interaction_type'] == 'Agonist':
        edge_color.extend(['yellow', 'yellow', 'yellow'])
    elif edge[2]['interaction_type'] == 'Inhibition':
        edge_color.extend(['purple', 'purple', 'purple'])
    elif edge[2]['interaction_type'] == 'others':
        edge_color.extend(['silver', 'silver', 'silver'])
    else:
        edge_color.extend(['rgb(125,125,125)', 'rgb(125,125,125)', 'rgb(125,125,125)'])

# Create node trace for plotting
node_x = []
node_y = []
node_z = []
node_text = []
node_color = []  # New list for node colors based on type

for node in G.nodes():
    x, y, z = node_positions[node]
    node_x.append(x)
    node_y.append(y)
    node_z.append(z)
    node_text.append(node)

    # Assign node colors based on type (chemical or gene)
    if node in df['ChemicalName'].values:
        node_color.append('rgb(0,255,0)')  # Green for chemicals
    else:
        node_color.append('rgb(255,0,0)')  # Red for genes

# Create 3D plot
fig = go.Figure()

# Add edges
fig.add_trace(go.Scatter3d(
    x=edge_x, y=edge_y, z=edge_z,
    mode='lines',
    line=dict(color=edge_color, width=1),
    hoverinfo='text',
    text=edge_text,
    name='Interactions',
    hovertext=edge_text  # Show edge interaction type on hover
))

# Add nodes
fig.add_trace(go.Scatter3d(
    x=node_x, y=node_y, z=node_z,
    mode='markers',
    marker=dict(symbol='circle', size=5, color=node_color),
    text=node_text,
    hoverinfo='text',
    name='Nodes',
    hovertext=node_text,  # Show node labels on hover
    textposition='top center',  # Display node labels above nodes
))

# Set layout
fig.update_layout(
    title='Chemical-Gene Interaction Knowledge Graph (Top 1160)',
    showlegend=True,
    scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False),
    ),
    legend=dict(
        x=0.85,
        y=0.9,
        traceorder='normal',
        font=dict(size=12,),
        bgcolor='rgba(255, 255, 255, 0.6)',
        bordercolor='rgba(0, 0, 0, 0.1)',
        borderwidth=2
    )
)

# Add legend items manually
fig.add_trace(go.Scatter3d(
    x=[None],
    y=[None],
    mode='markers',
    marker=dict(symbol='circle', size=10, color='rgb(0,255,0)'),
    showlegend=True,
    name='Chemical Nodes'
))

fig.add_trace(go.Scatter3d(
    x=[None],
    y=[None],
    mode='markers',
    marker=dict(symbol='circle', size=10, color='rgb(255,0,0)'),
    showlegend=True,
    name='Gene Nodes'
))

fig.add_trace(go.Scatter3d(
    x=[None],
    y=[None],
    mode='lines',
    line=dict(color='orange', width=2),
    showlegend=True,
    name='Proliferation Inhibition Edges'
))

fig.add_trace(go.Scatter3d(
    x=[None],
    y=[None],
    mode='lines',
    line=dict(color='pink', width=2),
    showlegend=True,
    name='Activation Edges'
))

fig.add_trace(go.Scatter3d(
    x=[None],
    y=[None],
    mode='lines',
    line=dict(color='yellow', width=2),
    showlegend=True,
    name='Agonist Edges'
))

fig.add_trace(go.Scatter3d(
    x=[None],
    y=[None],
    mode='lines',
    line=dict(color='purple', width=2),
    showlegend=True,
    name='Inhibition Edges'
))

fig.add_trace(go.Scatter3d(
    x=[None],
    y=[None],
    mode='lines',
    line=dict(color='silver', width=2),
    showlegend=True,
    name='Other Edges'
))

# Show the plot
import plotly.io as pio
pio.show(fig)

fig.write_html('/content/gene-Chemical_interaction_knowledge_graph(Top 1160).html')



In [23]:
import streamlit as st
import pandas as pd
import re
import networkx as nx
import plotly.graph_objects as go
# Load data
file_path = '/content/Task10_Gene_to_Chemical.csv'
df = pd.read_csv(file_path)

# Preprocess sentences
def preprocess_sentence(sentence):
    sentence = re.sub(r'[^\w\s]', '', sentence).lower()
    return sentence

df['Sentence'] = df['Sentence'].astype(str)
df['Sentence'] = df['Sentence'].apply(preprocess_sentence)
def generate_graph(selected_node):
    # Filter dataframe based on selected node
    filtered_df = df[(df['ChemicalName'] == selected_node) | (df['Genes'] == selected_node)]

    # Create a directed graph
    G = nx.DiGraph()

    # Add edges between chemicals and genes with interaction type as edge attribute
    for _, row in filtered_df.iterrows():
        G.add_edge(row['Chemical'], row['Gene'], interaction_type=row['Interaction type'])

    # Create a 3D spring layout
    pos = nx.spring_layout(G, dim=3)

    # Extract node positions for 3D plotting
    node_positions = {node: pos[node] for node in G.nodes}

    # Create edge trace for plotting
    edge_x = []
    edge_y = []
    edge_z = []
    edge_text = []
    edge_color = []

    for edge in G.edges(data=True):
        x0, y0, z0 = node_positions[edge[0]]
        x1, y1, z1 = node_positions[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])
        edge_text.append(edge[2]['interaction_type'])

        # Assign edge colors based on interaction type
        if edge[2]['interaction_type'] == 'Proliferation Inhibition':
            edge_color.extend(['orange', 'orange', 'orange'])
        elif edge[2]['interaction_type'] == 'Activation':
            edge_color.extend(['pink', 'pink', 'pink'])
        elif edge[2]['interaction_type'] == 'Agonist':
            edge_color.extend(['yellow', 'yellow', 'yellow'])
        elif edge[2]['interaction_type'] == 'Inhibition':
            edge_color.extend(['purple', 'purple', 'purple'])
        elif edge[2]['interaction_type'] == 'others':
            edge_color.extend(['silver', 'silver', 'silver'])
        else:
            edge_color.extend(['rgb(125,125,125)', 'rgb(125,125,125)', 'rgb(125,125,125)'])

    # Create node trace for plotting
    node_x = []
    node_y = []
    node_z = []
    node_text = []
    node_color = []

    for node in G.nodes():
        x, y, z = node_positions[node]
        node_x.append(x)
        node_y.append(y)
        node_z.append(z)
        node_text.append(node)

        # Assign node colors based on type
        if node in df['Chemicals'].values:
            node_color.append('rgb(0,255,0)')  # Green for chemicals
        else:
            node_color.append('rgb(255,0,0)')  # Red for genes

    # Create 3D plot
    fig = go.Figure()

    # Add edges
    fig.add_trace(go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        mode='lines',
        line=dict(color=edge_color, width=1),
        hoverinfo='text',
        text=edge_text,
        name='Interactions',
        hovertext=edge_text
    ))

    # Add nodes
    fig.add_trace(go.Scatter3d(
        x=node_x, y=node_y, z=node_z,
        mode='markers',
        marker=dict(symbol='circle', size=5, color=node_color),
        text=node_text,
        hoverinfo='text',
        name='Nodes',
        hovertext=node_text,
        textposition='top center'
    ))

    # Set layout
    fig.update_layout(
        title=f'Gene-Chemical Interaction Graph for {selected_node}',
        showlegend=True,
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False),
        ),
        legend=dict(
            x=0.85,
            y=0.9,
            traceorder='normal',
            font=dict(size=12,),
            bgcolor='rgba(255, 255, 255, 0.6)',
            bordercolor='rgba(0, 0, 0, 0.1)',
            borderwidth=2
        )
    )

    return fig
def generate_graph(selected_node):
    # Filter dataframe based on selected node
    filtered_df = df[(df['ChemicalName'] == selected_node) | (df['Genes'] == selected_node)]

    # Create a directed graph
    G = nx.DiGraph()

    # Add edges between chemicals and genes with interaction type as edge attribute
    for _, row in filtered_df.iterrows():
        G.add_edge(row['ChemicalName'], row['Genes'], interaction_type=row['Interaction type'])

    # Create a 3D spring layout
    pos = nx.spring_layout(G, dim=3)

    # Extract node positions for 3D plotting
    node_positions = {node: pos[node] for node in G.nodes}

    # Create edge trace for plotting
    edge_x = []
    edge_y = []
    edge_z = []
    edge_text = []
    edge_color = []

    for edge in G.edges(data=True):
        x0, y0, z0 = node_positions[edge[0]]
        x1, y1, z1 = node_positions[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])
        edge_text.append(edge[2]['interaction_type'])

        # Assign edge colors based on interaction type
        if edge[2]['interaction_type'] == 'Proliferation Inhibition':
            edge_color.extend(['orange', 'orange', 'orange'])
        elif edge[2]['interaction_type'] == 'Activation':
            edge_color.extend(['pink', 'pink', 'pink'])
        elif edge[2]['interaction_type'] == 'Agonist':
            edge_color.extend(['yellow', 'yellow', 'yellow'])
        elif edge[2]['interaction_type'] == 'Inhibition':
            edge_color.extend(['purple', 'purple', 'purple'])
        elif edge[2]['interaction_type'] == 'others':
            edge_color.extend(['silver', 'silver', 'silver'])
        else:
            edge_color.extend(['rgb(125,125,125)', 'rgb(125,125,125)', 'rgb(125,125,125)'])

    # Create node trace for plotting
    node_x = []
    node_y = []
    node_z = []
    node_text = []
    node_color = []

    for node in G.nodes():
        x, y, z = node_positions[node]
        node_x.append(x)
        node_y.append(y)
        node_z.append(z)
        node_text.append(node)

        # Assign node colors based on type
        if node in df['ChemicalName'].values:
            node_color.append('rgb(0,255,0)')  # Green for chemicals
        else:
            node_color.append('rgb(255,0,0)')  # Red for genes

    # Create 3D plot
    fig = go.Figure()

    # Add edges
    fig.add_trace(go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        mode='lines',
        line=dict(color=edge_color, width=1),
        hoverinfo='text',
        text=edge_text,
        name='Interactions',
        hovertext=edge_text
    ))

    # Add nodes
    fig.add_trace(go.Scatter3d(
        x=node_x, y=node_y, z=node_z,
        mode='markers',
        marker=dict(symbol='circle', size=5, color=node_color),
        text=node_text,
        hoverinfo='text',
        name='Nodes',
        hovertext=node_text,
        textposition='top center'
    ))

    # Set layout
    fig.update_layout(
        title=f'Chemical-Gene Interaction Graph for {selected_node}',
        showlegend=True,
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False),
        ),
        legend=dict(
            x=0.85,
            y=0.9,
            traceorder='normal',
            font=dict(size=12,),
            bgcolor='rgba(255, 255, 255, 0.6)',
            bordercolor='rgba(0, 0, 0, 0.1)',
            borderwidth=2
        )
    )

    return fig
# Streamlit app
def main():
    st.title('Gene-Chemical Interaction Explorer')

    selected_node = st.selectbox('Select a node:', df['ChemicalName'].unique().tolist() + df['Genes'].unique().tolist())

    st.write(f"You selected: {selected_node}")

    st.plotly_chart(generate_graph(selected_node), use_container_width=True)

if __name__ == "__main__":
    main()
